In [1]:
import numpy as np # linear algebra
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

train_data = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv', index_col=0)
test_data = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv', index_col=0)


In [2]:
train_data.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [3]:
missing = train_data.isna()
percent = (missing.sum()/missing.count()*100).sort_values(ascending=False)
missing_columns = percent[percent > 0].index.tolist()
print('Columns which have missing values: \n{0}'.format(missing_columns))

Columns which have missing values: 
['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 'LotFrontage', 'GarageYrBlt', 'GarageCond', 'GarageType', 'GarageFinish', 'GarageQual', 'BsmtExposure', 'BsmtFinType2', 'BsmtCond', 'BsmtQual', 'BsmtFinType1', 'MasVnrArea', 'MasVnrType', 'Electrical']


In [4]:
#train_data.drop(missing_columns, axis=1, inplace=True)
test_data.drop(missing_columns, axis=1, inplace=True)

In [5]:
X = train_data[['LotArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
                'TotalBsmtSF', '1stFlrSF','2ndFlrSF','LowQualFinSF', 'GrLivArea', 'WoodDeckSF', 
                'OpenPorchSF', 'EnclosedPorch','3SsnPorch', 'ScreenPorch', 'PoolArea',
                'MSSubClass', 'OverallQual', 'OverallCond', 'BsmtFullBath', 'BsmtHalfBath', 
                'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 
                'GarageCars']]

y = train_data['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


regressor = LinearRegression()
regressor.fit(X_train, y_train)
#print(regressor.score(X_test, y_test))
y_pred1 = regressor.predict(X_train)
print(r2_score(y_train, y_pred1))
y_pred2 = regressor.predict(X_test)
print(r2_score(y_test, y_pred2))




0.7922486967049529
0.808158666578985


In [6]:
import numpy as np
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
sgdr = SGDRegressor(alpha=0.0001, epsilon=0.00001, eta0=0.00001,penalty='L2')
sgdr.fit(X_train, y_train)
score = sgdr.score(X_train, y_train)
print("R-squared:", score)





R-squared: -1.9400249411568348e+18


In [7]:
from sklearn.linear_model import Ridge
import numpy as np
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
clf = Ridge(alpha=0.001)
clf.fit(X_train, y_train)
ridge_predicted = clf.predict(X_train)

print(r2_score(y_train, ridge_predicted))


0.7938348755014997


In [8]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.001)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
clf.fit(X_train, y_train)
lasso_predicted = clf.predict(X_train)

print(r2_score(y_train, lasso_predicted))

0.8277522404513358


In [9]:
from sklearn.linear_model import ElasticNet
regr = ElasticNet(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
regr.fit(X_train, y_train)
elastic_predicted = regr.predict(X_train)
print(r2_score(y_train, elastic_predicted))


0.7797324800057094


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 995722979047.0709, tolerance: 824359758.2541426
  positive)


In [10]:
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(64, activation='relu'))
model.add(Dense(units = 64, activation = "relu"))
model.add(layers.Dense(1))
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model.fit(X_train,y_train,batch_size=64,epochs=600, validation_data=(X_test,y_test))
model.summary()

2022-02-24 23:11:32.572328: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-02-24 23:11:32.711930: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/600
19/19 [==============================] - 1s 16ms/step - loss: 37065867264.0000 - mae: 176552.3438 - val_loss: 38157516800.0000 - val_mae: 175973.9375
Epoch 2/600
19/19 [==============================] - 0s 5ms/step - loss: 33260058624.0000 - mae: 166325.1719 - val_loss: 33953476608.0000 - val_mae: 164660.5000
Epoch 3/600
19/19 [==============================] - 0s 5ms/step - loss: 28900814848.0000 - mae: 153560.1250 - val_loss: 28739188736.0000 - val_mae: 149355.3906
Epoch 4/600
19/19 [==============================] - 0s 5ms/step - loss: 24089466880.0000 - mae: 137516.3438 - val_loss: 23169980416.0000 - val_mae: 130906.3438
Epoch 5/600
19/19 [==============================] - 0s 5ms/step - loss: 19778260992.0000 - mae: 120715.7812 - val_loss: 17802729472.0000 - val_mae: 110414.0156
Epoch 6/600
19/19 [==============================] - 0s 4ms/step - loss: 16442334208.0000 - mae: 103018.2109 - val_loss: 13749131264.0000 - val_mae: 93248.9531
Epoch 7/600
19/19 [===============

In [11]:
y_pred1 = model.predict(X_train)
print(r2_score(y_train, y_pred1))

0.654529383013809


In [12]:
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(19,activation='relu'))
model.add(Dense(19,activation='relu'))
model.add(Dense(19,activation='relu'))
model.add(Dense(19,activation='relu'))
model.add(Dense(1))
model.compile(optimizer='Adam',loss='mse')


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
model.fit(X_train,y_train,batch_size=128,epochs=100, validation_data=(X_test,y_test))
model.summary()


Epoch 1/100
10/10 [==============================] - 1s 21ms/step - loss: 38995341312.0000 - val_loss: 39100710912.0000
Epoch 2/100
10/10 [==============================] - 0s 6ms/step - loss: 38664138752.0000 - val_loss: 38769938432.0000
Epoch 3/100
10/10 [==============================] - 0s 6ms/step - loss: 38288949248.0000 - val_loss: 38321881088.0000
Epoch 4/100
10/10 [==============================] - 0s 6ms/step - loss: 37741109248.0000 - val_loss: 37656842240.0000
Epoch 5/100
10/10 [==============================] - 0s 7ms/step - loss: 36932038656.0000 - val_loss: 36636401664.0000
Epoch 6/100
10/10 [==============================] - 0s 6ms/step - loss: 35707240448.0000 - val_loss: 35105255424.0000
Epoch 7/100
10/10 [==============================] - 0s 6ms/step - loss: 33918220288.0000 - val_loss: 32890996736.0000
Epoch 8/100
10/10 [==============================] - 0s 7ms/step - loss: 31388086272.0000 - val_loss: 29663698944.0000
Epoch 9/100
10/10 [============================

In [14]:
y_pred1 = model.predict(X_train)
print(r2_score(y_train, y_pred1))

0.625036035743497
